In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/C247Project/project/

/content/drive/.shortcut-targets-by-id/1lO_lCKfnzu9fB3MDutIRFUOktcz9B9el/C247Project/project


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Permute, Bidirectional, BatchNormalization, Conv2D, LSTM, Dense, Dropout, ELU, Flatten, MaxPool2D, TimeDistributed, Dense
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import lecun_uniform
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

## Import Datasets

In [ ]:
X_test = np.load("Dataset/X_test.npy")
y_test = np.load("Dataset/y_test.npy")
X_train_valid = np.load("Dataset/X_train_valid.npy")
y_train_valid = np.load("Dataset/y_train_valid.npy")
person_train_valid = np.load("Dataset/person_train_valid.npy")
person_test = np.load("Dataset/person_test.npy")
X_synthetic = np.load("synthetic_data.npy")
y_synthetic = y_train_valid

## Adjusting the labels 
y_train_valid -= 769
y_test -= 769

# Constant
num_classes = 4

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
print ('Synthetic data shape: {}'.format(X_synthetic.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)
Synthetic data shape: (2115, 22, 1000)


## Function Definitions

In [ ]:
'''
    Preprocess data: 
    - Trim
    - Maxpool
    - Average + noise
    - Subsampling + noise
'''
def data_prep(X,y,sub_sample,average,noise):
    total_X = None
    total_y = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,500)
    X = X[:,:,0:500]

    # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    total_X, total_y = X_max, y
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    
    # Subsampling
    for i in range(sub_sample):
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
    
    return total_X,total_y


'''
    Random split training data into training and validation set
    Reshape training, validation, and test sets
'''
def rand_split_reshape_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep, valid_size=1500):
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(len(X_train_valid_prep), valid_size, replace=False)
    ind_train = np.array(list(set(range(len(X_train_valid_prep))).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test_prep, 4)

    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    return x_train, y_train, x_valid, y_valid, x_test, y_test
    

'''
    Helper function
    Random split training data into training and validation set
'''
def rand_split_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep, valid_size=1500):
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(len(X_train_valid_prep), valid_size, replace=False)
    ind_train = np.array(list(set(range(len(X_train_valid_prep))).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid]
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]

    return x_train, y_train, x_valid, y_valid, X_test_prep, y_test_prep


'''
    Helper function
    Convert labels (y) into categorical
'''
def convert_categorical(x_train, y_train, x_valid, y_valid, X_test_prep, y_test):
    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test, 4)
    return x_train, y_train, x_valid, y_valid, X_test_prep, y_test


'''
    Helper function
    Reshape training, validation, and test data
'''
def reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test): 
    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    return x_train, y_train, x_valid, y_valid, x_test, y_test


'''
    Concatenate synthetic data into the training dataset
'''
def add_synthetic_data(x_train_valid, y_train_valid):
    # Stack original and synthetic data
    x_train_valid = np.vstack((x_train_valid, X_synthetic))
    y_train_valid = np.hstack((y_train_valid, y_synthetic))
    
    return x_train_valid, y_train_valid


'''
    Selects entries corresponding to selected person
'''
def sub_selection(idx, X_train, y_train, X_test, y_test, person_train_valid, person_test):
    train_idx = np.where(person_train_valid == idx)[0]
    test_idx = np.where(person_test == idx)[0]
    X_train = X_train[train_idx]
    y_train = y_train[train_idx]
    X_test = X_test[test_idx]
    y_test = y_test[test_idx]
    return X_train, y_train, X_test, y_test


'''
    Train model by (1) preprocess data, (2) split into training and validation sets
'''
def train(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)

    # Preprocess Data: Training and testing data
    X_train_valid_prep, y_train_valid_prep = data_prep(X_train_valid,y_train_valid,2,2,True)
    X_test_prep, y_test_prep = data_prep(X_test,y_test,2,2,True)

    # Randomly split and reshape: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_reshape_data(X_train_valid_prep, y_train_valid_prep, X_test_prep, y_test_prep)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score


'''
    Train model by (1) split into training and validation sets, (2) preprocess data
'''
def train_split_first(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)
    
    train_valid_size = len(X_train_valid)
    valid_size = int(train_valid_size * 0.17)

    # Randomly split: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_data(X_train_valid, y_train_valid, X_test, y_test, valid_size)

    # Preprocess: Training, validation, and testing data
    x_train, y_train = data_prep(x_train,y_train,2,2,True)
    x_valid, y_valid = data_prep(x_valid, y_valid,2,2,True)
    x_test, y_test = data_prep(x_test, y_test,2,2,True)

    # Convert labels into categorical
    x_train, y_train, x_valid, y_valid, x_test, y_test = convert_categorical(x_train, y_train, x_valid, y_valid, x_test, y_test)

    # Reshape: Training, validation, and testing data
    x_train, y_train, x_valid, y_valid, x_test, y_test = reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score

'''
    Train model by (1) Add synthetic data, (2) split into training and validation sets, (3) preprocess data
'''
def train_split_first_with_synthetic(model, params, data, idx=None):
    epoch_num, batch_size = params
    X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test = data

    # Add synthetic data
    X_train_valid, y_train_valid = add_synthetic_data(X_train_valid, y_train_valid)

    # Subject split
    if idx is not None:
        X_train_valid, y_train_valid, X_test, y_test = sub_selection(idx, X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test)

    train_valid_size = len(X_train_valid)
    valid_size = int(train_valid_size * 0.18)

    # Randomly split: Training Data 
    x_train, y_train, x_valid, y_valid, x_test, y_test = rand_split_data(X_train_valid, y_train_valid, X_test, y_test, valid_size)

    # Preprocess: Training, validation, and testing data
    x_train, y_train = data_prep(x_train,y_train,2,2,True)
    x_valid, y_valid = data_prep(x_valid, y_valid,2,2,True)
    x_test, y_test = data_prep(x_test, y_test,2,2,True)

    # Cnvert labels into categorical
    x_train, y_train, x_valid, y_valid, x_test, y_test = convert_categorical(x_train, y_train, x_valid, y_valid, x_test, y_test)

    # Reshape: Training, validation, and testing data
    x_train, y_train, x_valid, y_valid, x_test, y_test = reshape_data(x_train, y_train, x_valid, y_valid, x_test, y_test)

    model.fit(x_train, y_train, epochs=epoch_num, batch_size=batch_size, 
              validation_data=(x_valid, y_valid), shuffle=True, verbose=1)
    train_score = model.evaluate(x_train, y_train)
    test_score = model.evaluate(x_test, y_test)

    print('train {:s}: {:.3f}%'.format(model.metrics_names[1], train_score[1]*100))
    print('test {:s}: {:.3f}%'.format(model.metrics_names[1], test_score[1]*100))
    
    return train_score, test_score 

## Model Setup

In [ ]:
def RNN_model(kernel_initializer, params):
    lr, dropout_rate = params
    channels_in_1, channels_out_1, kernel_size_1, stride_1 = (250,1,22), 25, (10, 1), 1
    pool_size_1, pool_stride_1 = (3, 1), 1
    channels_out_2, kernel_size_2, stride_2 = 50, (10, 1), 1
    pool_size_2, pool_stride_2 = (3, 1), 1
    channels_out_3, kernel_size_3, stride_3 = 100, (10, 1), 1
    pool_size_3, pool_stride_3 = (3, 1), 1
    channels_out_4, kernel_size_4, stride_4 = 200, (10, 1), 1
    pool_size_4, pool_stride_4 = (3, 1), 1
    hiddendim_1, hiddendim_2, hiddendim_3 = 100, 50, 25
    model = Sequential(
        [
        Conv2D(filters=channels_out_1, kernel_size=kernel_size_1, padding='same', activation='elu', input_shape=channels_in_1),
        MaxPool2D(pool_size=pool_size_1, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_2, kernel_size=kernel_size_2, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_2, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_3, kernel_size=kernel_size_3, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_3, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Conv2D(filters=channels_out_4, kernel_size=kernel_size_4, padding='same', activation='elu'),
        MaxPool2D(pool_size=pool_size_4, padding='same'),
        BatchNormalization(),
        Dropout(dropout_rate),
         
        Permute((2, 3, 1)),
        TimeDistributed(Flatten()),
        Bidirectional(LSTM(hiddendim_1, kernel_initializer=kernel_initializer, return_sequences=True)),
        Bidirectional(LSTM(hiddendim_2, kernel_initializer=kernel_initializer, return_sequences=True)),
        Bidirectional(LSTM(hiddendim_3, kernel_initializer=kernel_initializer)),
        Dropout(dropout_rate),

        Dense(num_classes, activation='softmax'),
        ]
    )
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(learning_rate=lr),
                  metrics=[categorical_accuracy])
    return model

## Hyper parameter tuning

#### (1) Preprocess (2) Split

In [ ]:
epoch_num = 40
lrs = [1e-3, 8e-4, 6e-4, 4e-4]
bzs = [30, 64, 80]
drs = [0.4, 0.5, 0.6]

tuning_results = []

for lr in lrs:
    for bz in bzs:
        for dr in drs:
            curr_model = RNN_model(lecun_uniform(seed=42), [lr, dr])
            train_acc, test_acc = train(curr_model,
                                        [epoch_num, bz],
                                        [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test]
                                        )
            tuning_results.append((lr, bz, dr, train_acc, test_acc))

Epoch 1/40
232/232 [==============================] - 18s 34ms/step - loss: 1.2681 - categorical_accuracy: 0.4017 - val_loss: 1.2110 - val_categorical_accuracy: 0.4227
Epoch 2/40
232/232 [==============================] - 5s 23ms/step - loss: 1.1390 - categorical_accuracy: 0.4947 - val_loss: 1.0845 - val_categorical_accuracy: 0.5287
Epoch 3/40
232/232 [==============================] - 5s 23ms/step - loss: 1.0401 - categorical_accuracy: 0.5534 - val_loss: 0.9567 - val_categorical_accuracy: 0.6067
Epoch 4/40
232/232 [==============================] - 5s 24ms/step - loss: 0.9439 - categorical_accuracy: 0.6118 - val_loss: 0.8529 - val_categorical_accuracy: 0.6673
Epoch 5/40
232/232 [==============================] - 5s 23ms/step - loss: 0.8523 - categorical_accuracy: 0.6677 - val_loss: 0.8168 - val_categorical_accuracy: 0.6733
Epoch 6/40
232/232 [==============================] - 5s 23ms/step - loss: 0.7654 - categorical_accuracy: 0.7056 - val_loss: 0.6174 - val_categorical_accuracy: 0.77

In [ ]:
for res in tuning_results:
    print("lr={", res[0], "}", end=" ")
    print("bz={", res[1], "}", end=" ")
    print("dr={", res[2], "}", end=" ")
    print('train: {:.3f}%'.format(res[3][1]*100), end=" ")
    print('test: {:.3f}%'.format(res[4][1]*100))

lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 99.856% test: 58.804%
lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 99.181% test: 60.609%
lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 90.460% test: 61.625%
lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 99.914% test: 61.569%
lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 99.741% test: 59.819%
lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 93.491% test: 63.318%
lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 99.943% test: 62.472%
lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 99.770% test: 57.788%
lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 92.241% test: 60.666%
lr={ 0.0008 } bz={ 30 } dr={ 0.4 } train: 99.957% test: 58.183%
lr={ 0.0008 } bz={ 30 } dr={ 0.5 } train: 99.626% test: 58.973%
lr={ 0.0008 } bz={ 30 } dr={ 0.6 } train: 93.405% test: 63.657%
lr={ 0.0008 } bz={ 64 } dr={ 0.4 } train: 99.899% test: 63.318%
lr={ 0.0008 } bz={ 64 } dr={ 0.5 } train: 99.655% test: 60.892%
lr={ 0.0008 } bz={ 64 } dr={ 0.6 } train: 93.807% test: 60.158%
lr={ 0.0008 } bz={ 80 } dr={ 0.4 } train: 100.000

#### (1) Split (2) Preprocess

In [ ]:
epoch_num = 40
lrs = [1e-3, 8e-4, 6e-4, 4e-4]
bzs = [30, 64, 80]
drs = [0.4, 0.5, 0.6]

tuning_results = []

for lr in lrs:
    for bz in bzs:
        for dr in drs:
            curr_model = RNN_model(lecun_uniform(seed=42), [lr, dr])
            train_acc, test_acc = train_split_first(curr_model,
                                        [epoch_num, bz],
                                        [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test]
                                        )
            tuning_results.append((lr, bz, dr, train_acc, test_acc))

Epoch 1/40
235/235 [==============================] - 29s 35ms/step - loss: 1.2566 - categorical_accuracy: 0.4167 - val_loss: 1.2989 - val_categorical_accuracy: 0.4171
Epoch 2/40
235/235 [==============================] - 5s 23ms/step - loss: 1.0912 - categorical_accuracy: 0.5275 - val_loss: 1.2005 - val_categorical_accuracy: 0.4742
Epoch 3/40
235/235 [==============================] - 5s 23ms/step - loss: 0.9793 - categorical_accuracy: 0.6005 - val_loss: 1.1690 - val_categorical_accuracy: 0.5056
Epoch 4/40
235/235 [==============================] - 5s 23ms/step - loss: 0.8659 - categorical_accuracy: 0.6570 - val_loss: 1.1821 - val_categorical_accuracy: 0.5195
Epoch 5/40
235/235 [==============================] - 5s 23ms/step - loss: 0.7540 - categorical_accuracy: 0.7067 - val_loss: 1.1780 - val_categorical_accuracy: 0.5341
Epoch 6/40
235/235 [==============================] - 6s 24ms/step - loss: 0.6693 - categorical_accuracy: 0.7474 - val_loss: 1.2139 - val_categorical_accuracy: 0.56

In [ ]:
# Model, train_split_first
# epoch_num = 40
# lrs = [1e-3, 8e-4, 6e-4, 4e-4]
# bzs = [30, 64, 80]
# drs = [0.4, 0.5, 0.6]

# for res in tuning_results:
#     print("lr={", res[0], "}", end=" ")
#     print("bz={", res[1], "}", end=" ")
#     print("dr={", res[2], "}", end=" ")
#     print('train: {:.3f}%'.format(res[3][1]*100), end=" ")
#     print('test: {:.3f}%'.format(res[4][1]*100))

# BEST TEST ACCURACY

print('lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 100.000% test: 58.239%')
print('lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 99.915%  test: 59.537%')
print('lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 92.711%  test: 59.707%')
print('lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 57.901%')
print('lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 99.957% test: 61.230%')
print('lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 96.669%  test: 59.029%')
print('lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 100.000% test: 61.174%')
print('lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 99.986%  test: 62.754%')
print('lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 95.644%  test: 59.255%')
print('lr={ 0.0008 } bz={ 30 } dr={ 0.4 } train: 100.000% test: 56.885%')
print('lr={ 0.0008 } bz={ 30 } dr={ 0.5 } train: 99.972%  test: 56.941%')
print('lr={ 0.0008 } bz={ 30 } dr={ 0.6 } train: 94.063%  test: 54.966%')
print('lr={ 0.0008 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 56.603%')
print('lr={ 0.0008 } bz={ 64 } dr={ 0.5 } train: 99.972%  test: 58.239%')
print('lr={ 0.0008 } bz={ 64 } dr={ 0.6 } train: 95.914%  test: 57.844%')
print('lr={ 0.0008 } bz={ 80 } dr={ 0.4 } train: 99.986%  test: 60.327%')
print('lr={ 0.0008 } bz={ 80 } dr={ 0.5 } train: 99.886%  test: 57.280%')
print('lr={ 0.0008 } bz={ 80 } dr={ 0.6 } train: 97.765%  test: 62.472%')
print('lr={ 0.0006 } bz={ 30 } dr={ 0.4 } train: 100.000% test: 55.756%')
print('lr={ 0.0006 } bz={ 30 } dr={ 0.5 } train: 99.801% test: 57.844%')
print('lr={ 0.0006 } bz={ 30 } dr={ 0.6 } train: 95.886% test: 58.126%')
print('lr={ 0.0006 } bz={ 64 } dr={ 0.4 } train: 99.957% test: 60.779%')
print('lr={ 0.0006 } bz={ 64 } dr={ 0.5 } train: 99.900% test: 60.158%')
print('lr={ 0.0006 } bz={ 64 } dr={ 0.6 } train: 95.473% test: 60.440%')
print('lr={ 0.0006 } bz={ 80 } dr={ 0.4 } train: 100.000% test: 60.779%')
print('lr={ 0.0006 } bz={ 80 } dr={ 0.5 } train: 99.986% test: 60.948%')
print('lr={ 0.0006 } bz={ 80 } dr={ 0.6 } train: 93.394% test: 58.521%')
print('lr={ 0.0004 } bz={ 30 } dr={ 0.4 } train: 99.972% test: 61.456%')
print('lr={ 0.0004 } bz={ 30 } dr={ 0.5 } train: 99.843% test: 62.133%')
print('lr={ 0.0004 } bz={ 30 } dr={ 0.6 } train: 94.419% test: 58.860%')
print('lr={ 0.0004 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 61.964%')
print('lr={ 0.0004 } bz={ 64 } dr={ 0.5 } train: 99.758% test: 60.102%')
print('lr={ 0.0004 } bz={ 64 } dr={ 0.6 } train: 90.760% test: 59.312%')
print('lr={ 0.0004 } bz={ 80 } dr={ 0.4 } train: 99.972% test: 58.183%')


lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 100.000% test: 58.239%
lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 99.915%  test: 59.537%
lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 92.711%  test: 59.707%
lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 57.901%
lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 99.957% test: 61.230%
lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 96.669%  test: 59.029%
lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 100.000% test: 61.174%
lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 99.986%  test: 62.754%
lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 95.644%  test: 59.255%
lr={ 0.0008 } bz={ 30 } dr={ 0.4 } train: 100.000% test: 56.885%
lr={ 0.0008 } bz={ 30 } dr={ 0.5 } train: 99.972%  test: 56.941%
lr={ 0.0008 } bz={ 30 } dr={ 0.6 } train: 94.063%  test: 54.966%
lr={ 0.0008 } bz={ 64 } dr={ 0.4 } train: 100.000% test: 56.603%
lr={ 0.0008 } bz={ 64 } dr={ 0.5 } train: 99.972%  test: 58.239%
lr={ 0.0008 } bz={ 64 } dr={ 0.6 } train: 95.914%  test: 57.844%
lr={ 0.0008 } bz={ 80 } dr={ 0.4 } 

#### (1) Add synthetic data (2) Split (3) Preprocess

In [ ]:
epoch_num = 40
lrs = [1e-3, 6e-4, 2e-4]
bzs = [30, 64, 80]
drs = [0.4, 0.5, 0.6]

tuning_results = []

for lr in lrs:
    for bz in bzs:
        for dr in drs:
            curr_model = RNN_model(lecun_uniform(seed=42), [lr, dr])
            train_acc, test_acc = train_split_first_with_synthetic(curr_model,
                                        [epoch_num, bz],
                                        [X_train_valid, y_train_valid, X_test, y_test, person_train_valid, person_test]
                                        )
            tuning_results.append((lr, bz, dr, train_acc, test_acc))

Epoch 1/40
463/463 [==============================] - 26s 18ms/step - loss: 1.3306 - categorical_accuracy: 0.3283 - val_loss: 1.3298 - val_categorical_accuracy: 0.3377
Epoch 2/40
463/463 [==============================] - 6s 13ms/step - loss: 1.2724 - categorical_accuracy: 0.3817 - val_loss: 1.2940 - val_categorical_accuracy: 0.3735
Epoch 3/40
463/463 [==============================] - 6s 13ms/step - loss: 1.2334 - categorical_accuracy: 0.4044 - val_loss: 1.2747 - val_categorical_accuracy: 0.3962
Epoch 4/40
463/463 [==============================] - 6s 13ms/step - loss: 1.1951 - categorical_accuracy: 0.4346 - val_loss: 1.2521 - val_categorical_accuracy: 0.4182
Epoch 5/40
463/463 [==============================] - 6s 13ms/step - loss: 1.1571 - categorical_accuracy: 0.4519 - val_loss: 1.2586 - val_categorical_accuracy: 0.4192
Epoch 6/40
463/463 [==============================] - 6s 13ms/step - loss: 1.1179 - categorical_accuracy: 0.4724 - val_loss: 1.2823 - val_categorical_accuracy: 0.39

In [ ]:
for res in tuning_results:
    print("lr={", res[0], "}", end=" ")
    print("bz={", res[1], "}", end=" ")
    print("dr={", res[2], "}", end=" ")
    print('train: {:.3f}%'.format(res[3][1]*100), end=" ")
    print('test: {:.3f}%'.format(res[4][1]*100))

lr={ 0.001 } bz={ 30 } dr={ 0.4 } train: 65.040% test: 56.716%
lr={ 0.001 } bz={ 30 } dr={ 0.5 } train: 64.622% test: 57.393%
lr={ 0.001 } bz={ 30 } dr={ 0.6 } train: 55.816% test: 56.772%
lr={ 0.001 } bz={ 64 } dr={ 0.4 } train: 65.631% test: 58.465%
lr={ 0.001 } bz={ 64 } dr={ 0.5 } train: 64.932% test: 59.932%
lr={ 0.001 } bz={ 64 } dr={ 0.6 } train: 61.668% test: 56.038%
lr={ 0.001 } bz={ 80 } dr={ 0.4 } train: 65.927% test: 59.312%
lr={ 0.001 } bz={ 80 } dr={ 0.5 } train: 64.709% test: 58.296%
lr={ 0.001 } bz={ 80 } dr={ 0.6 } train: 58.807% test: 57.562%
lr={ 0.0006 } bz={ 30 } dr={ 0.4 } train: 65.336% test: 57.675%
lr={ 0.0006 } bz={ 30 } dr={ 0.5 } train: 65.264% test: 58.916%
lr={ 0.0006 } bz={ 30 } dr={ 0.6 } train: 61.113% test: 55.756%
lr={ 0.0006 } bz={ 64 } dr={ 0.4 } train: 66.085% test: 60.666%
lr={ 0.0006 } bz={ 64 } dr={ 0.5 } train: 64.882% test: 60.779%
lr={ 0.0006 } bz={ 64 } dr={ 0.6 } train: 61.358% test: 62.302%
lr={ 0.0006 } bz={ 80 } dr={ 0.4 } train: 65.898%